In [1]:
!pip install ultralytics
!pip install git+https://github.com/ultralytics/ultralytics.git@main

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 13.7 MB/s eta 0:00:00a 0:00:01


In [ ]:
import yaml
import os
import shutil
from random import seed, shuffle

Данные в датасете заранее аугментированы (случайно искажены). В датасете швов скорее всего придется делать аугментации самим.

Создание файла с настройками нейронки.

In [ ]:
data = {'train' :  '/kaggle/working/images/train',
        'val' :  '/kaggle/working/images/val',
        'test' :  '/kaggle/working/images/test',
        'nc': 5,
        'names': ['adj', 'int', 'geo', 'pro', 'non']
        }

# overwrite the data to the .yaml file
with open('/kaggle/working/dataset.yaml', 'w') as f:
    yaml.dump(data, f)

# read the content in .yaml file
with open('/kaggle/working/dataset.yaml', 'r') as f:
    hamster_yaml = yaml.safe_load(f)
    display(hamster_yaml)

Создание структуры папок, нужной для обучения YOLOv8.

In [ ]:
!mkdir /kaggle/working/labels
!mkdir /kaggle/working/images
!mkdir /kaggle/working/images/train
!mkdir /kaggle/working/images/test
!mkdir /kaggle/working/images/val
!mkdir /kaggle/working/labels/train
!mkdir /kaggle/working/labels/test
!mkdir /kaggle/working/labels/val

In [ ]:

# Установите начальное значение для генератора случайных чисел для воспроизводимости
seed(1)

# Пропорции разделения
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Путь к папке с изображениями
images_path = '/kaggle/input/welding-seams-dataset'
images = [os.path.join(images_path, file) for file in os.listdir(images_path) if file.endswith('.jpg')]

# Перемешивание списка изображений
shuffle(images)

# Список меток
labels = [os.path.splitext(image_path)[0]+'.txt' for image_path in images]

# Разделение на выборки
train_split = int(train_ratio * len(images))
val_split = int(val_ratio * len(images)) + train_split

train_images = images[:train_split]
val_images = images[train_split:val_split]
test_images = images[val_split:]

train_labels = labels[:train_split]
val_labels = labels[train_split:val_split]
test_labels = labels[val_split:]

# Функция для копирования изображений в целевые папки
def copy_objects(objects_list, target_folder):
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    for obj in objects_list:
        shutil.copy(obj, target_folder)
        shutil.copy(obj, target_folder)

# Копирование изображений в соответствующие папки
copy_objects(train_images, '/kaggle/working/images/train')
copy_objects(val_images, '/kaggle/working/images/val')
copy_objects(test_images, '/kaggle/working/images/test')

copy_objects(train_labels, '/kaggle/working/labels/train')
copy_objects(val_labels, '/kaggle/working/labels/val')
copy_objects(test_labels, '/kaggle/working/labels/test')

Обучение

In [ ]:
import wandb
wandb.login(key='6a6d221f38aac6d82e19bb1cf2c801309be13006')

In [2]:
from ultralytics import YOLO

model = YOLO("/kaggle/input/yolo8_500/other/500/2/best.pt")


In [ ]:
# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data="/kaggle/working/dataset.yaml", epochs=500)

# Evaluate the model's performance on the validation set
results = model.val()

# Perform object detection on an image using the model
results = model("/kaggle/input/welding-seams-dataset/1 (3).jpg")

# Export the model to ONNX format
success = model.export(format="onnx")

In [ ]:
model.val()

In [30]:
results = model("/kaggle/input/welding-seams-dataset/2 (13).jpg")
print(len(results))

for result in results:
    print(result.orig_img)


image 1/1 /kaggle/input/welding-seams-dataset/2 (13).jpg: 384x640 5 adjs, 7.5ms
Speed: 2.2ms preprocess, 7.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
1
[[[125 122 118]
  [131 128 124]
  [125 122 118]
  ...
  [106 109 114]
  [102 105 110]
  [100 103 108]]

 [[129 126 122]
  [136 133 129]
  [137 134 130]
  ...
  [101 104 109]
  [102 105 110]
  [103 106 111]]

 [[143 140 136]
  [145 142 138]
  [140 137 133]
  ...
  [107 110 115]
  [107 110 115]
  [107 110 115]]

 ...

 [[104 104 110]
  [112 112 118]
  [108 110 118]
  ...
  [ 56  59  67]
  [ 59  62  70]
  [ 59  62  70]]

 [[101 101 107]
  [103 103 109]
  [ 99 101 109]
  ...
  [ 56  59  67]
  [ 59  62  70]
  [ 58  61  69]]

 [[100 100 106]
  [ 95  95 101]
  [ 90  92 100]
  ...
  [ 57  60  68]
  [ 60  63  71]
  [ 59  62  70]]]


Сохранение обученной модели

In [ ]:
model.export(format="onnx")